In [0]:
%sql
use catalog retail_consumer_goods;
use retail_consumer_goods.conversational_commerce_search;

In [ ]:
%sql
select * from customers where name = 'Chad Sanders'

In [ ]:
%sql
SELECT 
    p.name as product_name,
    p.price,
    i.interaction_type,
    i.timestamp,
    p.category
FROM retail_consumer_goods.conversational_commerce_search.interactions i
JOIN retail_consumer_goods.conversational_commerce_search.products p 
    ON i.product_id = p.product_id
WHERE i.customer_id = 'cust_091'
ORDER BY i.timestamp DESC
LIMIT 5

In [ ]:
import time

start = time.time()
df = spark.sql("""
    SELECT * FROM get_smart_product_recommendations(
        'Maria Shaw', 
        'ian49@example.org', 
        'home decor items', 
        NULL, 
        NULL, 
        150
    )
""")
count = df.count()
duration = time.time() - start

print(f"⏱️  Execution time: {duration:.2f} seconds")
print(f"📦 Found {count} products")
display(df)

In [0]:
%sql
-- Test Case: Updated function with email parameter for precise customer identification
-- This demonstrates the new email parameter functionality
SELECT 
    customer_name,
    customer_email,
    product_name,
    category,
    subcategory,
    brand,
    price,
    recommendation_score,
    recommendation_reason
FROM get_smart_product_recommendations("Peter Bryant", "heidi18@example.net", "fictional books", "penguin", NULL, 20)
ORDER BY price ASC;


In [0]:
%sql
SELECT * FROM validate_customer_by_name("Casey Romero");

In [0]:
%sql
SELECT * FROM validate_customer_by_name("Kristin Wyatt DVM");

In [0]:
%sql
-- Test Case A: Your exact use case - "show me all the outdoor gears ordered by Kristin"
SELECT 
    customer_name,
    customer_email,
    product_name,
    category,
    subcategory,
    brand,
    price,
    recommendation_score,
    recommendation_reason
FROM get_smart_product_recommendations("Susan Martin", NULL, "books for vacation", NULL, NULL, NULL)
order by price asc;    


In [0]:
%sql
SELECT 
    customer_name,
    product_name,
    category,
    subcategory,
    brand,
    price,
    recommendation_score,
    recommendation_reason,
    brand_match_score,
    price_in_range
FROM get_smart_product_recommendations("David Hawkins", NULL, "outdoor gears", NULL, NULL, NULL)
order by price asc;


In [0]:
%sql
-- Test Case C: Price range filtering - "electronics less than 1000 $"
SELECT 
    customer_name,
    product_name,
    category,
    subcategory,
    brand,
    price,
    recommendation_score,
    recommendation_reason,
    price_in_range
FROM get_smart_product_recommendations("Raymond Roberts", NULL, "gaming laptop", "HP", NULL, 1000);


In [0]:
%sql
-- Test Case F: General recommendations without any filters (original behavior)
SELECT 
    customer_name,
    product_name,
    category,
    subcategory,
    brand,
    price,
    recommendation_score,
    recommendation_reason
FROM get_smart_product_recommendations("Joann Willis", NULL, "laptops", NULL, NULL, NULL);


In [0]:
%sql
-- Test Case F: General recommendations without any filters (original behavior)
SELECT 
    customer_name,
    product_name,
    category,
    subcategory,
    brand,
    price,
    recommendation_score,
    recommendation_reason
FROM get_smart_product_recommendations("Miss Lori Tucker", NULL, NULL, NULL, NULL, NULL);


In [0]:
%sql
-- Test Case 5: General product search by description (keeping the existing function)
-- This is useful for general product discovery
SELECT 
    *
FROM product_lookup_by_description("outdoor camping gear");


In [0]:
%sql
select * from product_lookup_by_description("Pottery Barn Furniture Above");


In [0]:
%sql
-- Test the random customer lookup function
-- This will return a different random customer each time it's executed
SELECT * FROM get_random_customer_info();


In [0]:
%sql
-- Test multiple random customer selections
-- This demonstrates getting different random customers in sequence
SELECT 'First Random Customer' as selection_type, * FROM get_random_customer_info()
UNION ALL
SELECT 'Second Random Customer' as selection_type, * FROM get_random_customer_info()
UNION ALL  
SELECT 'Third Random Customer' as selection_type, * FROM get_random_customer_info();


In [0]:
%sql
-- Test Case A: Customer lookup by name (original functionality)
-- This will show the 10 most recent interactions for Charles Martinez
SELECT * FROM get_customer_interactions("Anthony Anderson", "kimberly35@example.com");


In [0]:
%sql
-- Test Case B: Customer lookup by email (NEW functionality)
-- This demonstrates exact email matching
SELECT 
    customer_name,
    customer_email,
    product_name,
    product_price,
    interaction_type,
    interaction_timestamp,
    customer_rating,
    days_since_interaction
FROM get_customer_interactions(NULL, "tcontreras@example.net");


In [ ]:
%sql
SELECT 
            customer_id,
            customer_name,
            customer_email,
            customer_age,
            customer_gender,
            customer_location,
            persona,
            preferred_categories,
            total_orders,
            total_spent
        FROM (
            SELECT 
                customer_id,
                name as customer_name,
                email as customer_email,
                age as customer_age,
                gender as customer_gender,
                location as customer_location,
                persona,
                preferred_categories,
                total_orders,
                total_spent,
                ROW_NUMBER() OVER (PARTITION BY persona ORDER BY RAND()) as row_num
            FROM retail_consumer_goods.conversational_commerce_search.customers
        ) ranked
        WHERE row_num <= 6
        ORDER BY persona, row_num;

In [ ]:
%sql
WITH ranked AS (
  SELECT
    customer_id,
    name AS customer_name,
    email AS customer_email,
    age AS customer_age,
    gender AS customer_gender,
    location AS customer_location,
    persona,
    preferred_categories,
    total_orders,
    total_spent,
    ROW_NUMBER() OVER (
      PARTITION BY persona
      ORDER BY hash(customer_id)
    ) AS row_num
  FROM retail_consumer_goods.conversational_commerce_search.customers
)

SELECT customer_id,
            customer_name,
            customer_email,
            customer_age,
            customer_gender,
            customer_location,
            persona,
            preferred_categories,
            total_orders,
            total_spent
FROM ranked
WHERE row_num <= 6
ORDER BY persona, row_num;

